In [1]:
# Import packages
import numpy as np
import cvxpy as cp
import mosek
import time
from sklearn.model_selection import train_test_split

import phi_divergence as phi
import robust_sampling as rs
import dataio
import util

C:\Users\JSTARRE\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def generate_data(random_seed, k, N):
    np.random.seed(random_seed)
    data = np.random.uniform(-1,1,size = (N,k)) # generates N random scenarios    
    return data 

def generate_data_with_nominal(random_seed, k, N):
    data_nominal = np.array([[0] * k])
    np.random.seed(random_seed)
    data = np.random.uniform(-1,1,size = (N-1,k)) # generate N-1 scenarios
    data = np.concatenate((data_nominal,data)) # add nominal case to training data
    return data

def solve_SCP(k, S, time_limit):
    #k = S.shape[1]
    x = cp.Variable(k, nonneg = True)
    if len(S) > 0:
        constraints = [-(1-cp.exp(-(S @ x))) <= 0, x <= 1]
    else:
        constraints = []
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    try:
        prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    except cp.error.SolverError:
        return (None, None)
    return (x.value, prob.value)

def uncertain_constraint(S, x):
    return -(1-np.exp(-(np.dot(S,x))))

def check_robust(bound, numeric_precision, beta=0):
    return (bound <= beta + numeric_precision)

def get_true_exp(x, k):
    true_exp = 1 / (2**k)
    for i in range(k):
        true_exp = true_exp * (1/x[i]) * (np.exp(x[i]) - np.exp(-x[i]))
    return true_exp - 1
    
def solve_toyproblem_true_exp(beta, k):
    x = cp.Variable(k, nonneg = True)
    s = cp.Variable(k, nonneg = False)
    constraints = [cp.sum(-cp.log(x)) + x + s <= cp.multiply(k,np.log(2))]
    constraints_2 = [cp.exp(-s[i]) - cp.exp(cp.multiply(-2,x[i]) - s[i]) <= 1 for i in range(k)]
    constraints.extend(constraints_2)
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(x.value, prob.value)

In [6]:
# Set parameter values
random_seed = 1
risk_measure = 'exp_constraint_leq' # options: 'chance_constraint', 'exp_constraint_leq', 'exp_constraint_geq'
num_obs_per_bin = 10
alpha = 0.10
beta = 0
k = 10
N_train = 250
N_test = 250
phi_div = phi.mod_chi2_cut
phi_dot = 2
numeric_precision = 1e-6 # To correct for floating-point math operations

In [7]:
# Get generated data
N_total = N_train + N_test

data = generate_data(random_seed, k, N_total)
data_train, data_test = train_test_split(data, train_size=(N_train/N_total), random_state=random_seed)

In [8]:
#OPTIONAL:
N_eval = 1000000
data_eval = generate_data(random_seed + 99, k, N_eval)

In [9]:
time_limit_search = 1*60 # in seconds (time provided to search algorithm)
time_limit_mosek = 10*60 # in seconds (for larger MIP / LP solves)
time_limit_solve = 5*60 # in seconds (for individuals solves of SCP)
max_nr_solutions = 1 # for easy problems with long time limits, we may want extra restriction
add_remove_threshold = 0.00 # This determines when randomness is introduced in add/removal decision
use_tabu = False # Determines whether the tabu list are used in the search

add_strategy = 'random_vio'
remove_strategy = 'random_active'
clean_strategy = (1000, 'all_inactive')


(runtime, num_iter, solutions, 
 best_sol, pareto_solutions) = rs.gen_and_eval_alg(data_train, data_test, beta, alpha, time_limit_search, time_limit_solve, 
                                                    max_nr_solutions, add_strategy, remove_strategy, clean_strategy, 
                                                    add_remove_threshold, use_tabu,
                                                    phi_div, phi_dot, numeric_precision,
                                                    solve_SCP, uncertain_constraint, check_robust,
                                                    risk_measure, random_seed, num_obs_per_bin, data_eval)

---------------------------------------------
iter: 1
b_train: 7.3060472394054985
b_test: 11.901011587082845
p_eval: 2.398209158669467
---------------------------------------------
iter: 2
b_train: 7.700590038815986
b_test: 6.34300968601957
p_eval: 2.150421412315096
---------------------------------------------
iter: 3
b_train: 5.059517598166101
b_test: 4.9897202846463005
p_eval: 1.4532300966042497
---------------------------------------------
iter: 4
b_train: 5.005956967773475
b_test: 9.061750817783206
p_eval: 1.4329138676589186
---------------------------------------------
iter: 5
b_train: 5.541690338217543
b_test: 11.561946111169668
p_eval: 1.4493028553749252
---------------------------------------------
iter: 6
b_train: 2.5775478395904017
b_test: 3.2822758684525435
p_eval: 0.7799931801646899
---------------------------------------------
iter: 7
b_train: 0.6223756572621779
b_test: 0.7880367036770339
p_eval: 0.22578101548645102
---------------------------------------------
iter: 8
b_

In [55]:
len(best_sol['scenario_set'])

54

In [18]:
get_true_exp(best_sol['sol'], k)

C:\Users\JSTARRE\AppData\Local\Temp\ipykernel_17384\761720593.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  true_exp = true_exp * (1/x[i]) * (np.exp(x[i]) - np.exp(-x[i]))
C:\Users\JSTARRE\AppData\Local\Temp\ipykernel_17384\761720593.py:37: RuntimeWarning: invalid value encountered in double_scalars
  true_exp = true_exp * (1/x[i]) * (np.exp(x[i]) - np.exp(-x[i]))


nan

In [ ]:
x_true, obj_true = solve_toyproblem_true_exp(beta, k)